In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, models
from PIL import Image, ImageEnhance
import wandb
import numpy as np
from tqdm import tqdm
import random

class DriftConfig:
    DATA_DIR = "/content/drive/MyDrive/tiny-imagenet-200"
    MODEL_PATH = "outputs/best_model.pth"

    # W&B Configuration - Can be set via environment variables
    WANDB_ENTITY = os.environ.get("WANDB_ENTITY")
    WANDB_PROJECT = os.environ.get("WANDB_PROJECT")
    WANDB_ARTIFACT = os.environ.get("WANDB_ARTIFACT")

    # Test subset size
    TEST_SUBSET_SIZE = 2000

    # Drift parameters
    BRIGHTNESS_FACTORS = [0.3, 0.5, 0.7, 1.0, 1.3, 1.5, 1.7]
    NOISE_LEVELS = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

    # Training hyperparameters
    BATCH_SIZE = 64
    NUM_WORKERS = 4
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Model
    MODEL_NAME = "resnet18"
    NUM_CLASSES = 200

    # Alert threshold
    ACCURACY_DROP_THRESHOLD = 5.0

def download_model_from_wandb(config):

    if os.path.exists(config.MODEL_PATH):
        print(f"\nModel already exists at: {config.MODEL_PATH}")
        return True

    try:
        print("\nLogging into W&B...")
        wandb.login()

        api = wandb.Api()
        artifact_path = f"{config.WANDB_ENTITY}/{config.WANDB_PROJECT}/{config.WANDB_ARTIFACT}"
        print(f"\nDownloading artifact: {artifact_path}")

        artifact = api.artifact(artifact_path)
        artifact_dir = artifact.download(root="outputs")

        print(f"Model downloaded successfully to: {artifact_dir}")

        if os.path.exists(config.MODEL_PATH):
            print(f"Model file verified at: {config.MODEL_PATH}")
            return True
        else:
            for file in os.listdir("outputs"):
                if file.endswith(".pth") or file.endswith(".pt"):
                    print(f"Found model file: {file}")
                    return True
            print("Model file not found in downloaded artifact")
            return False

    except Exception as e:
        print(f"\nError downloading model from W&B: {e}")
        return False

class TinyImageNetDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, split='val', transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.samples = []
        self.class_to_idx = {}

        if split == 'val' or split == 'test':
            self._load_val_data()

    def _load_val_data(self):
        val_dir = os.path.join(self.root_dir, 'val')

        if not os.path.exists(val_dir):
            raise FileNotFoundError(f"Val directory not found: {val_dir}")

        train_dir = os.path.join(self.root_dir, 'train')
        classes = sorted([d for d in os.listdir(train_dir)
                         if os.path.isdir(os.path.join(train_dir, d))])
        for idx, class_name in enumerate(classes):
            self.class_to_idx[class_name] = idx

        val_classes = [d for d in os.listdir(val_dir)
                      if os.path.isdir(os.path.join(val_dir, d)) and d in self.class_to_idx]

        if len(val_classes) > 0:
            print(f"Val set has class folder structure with {len(val_classes)} classes")
            for class_name in val_classes:
                class_idx = self.class_to_idx[class_name]
                class_dir = os.path.join(val_dir, class_name, 'images')

                if not os.path.exists(class_dir):
                    class_dir = os.path.join(val_dir, class_name)

                if os.path.exists(class_dir):
                    for img_name in os.listdir(class_dir):
                        if img_name.lower().endswith(('.jpeg', '.jpg', '.png')):
                            img_path = os.path.join(class_dir, img_name)
                            self.samples.append((img_path, class_idx))
        else:
            val_annotations = os.path.join(val_dir, 'val_annotations.txt')

            if os.path.exists(val_annotations):
                with open(val_annotations, 'r') as f:
                    for line in f:
                        parts = line.strip().split('\t')
                        img_name = parts[0]
                        class_name = parts[1]
                        img_path = os.path.join(val_dir, 'images', img_name)

                        if not os.path.exists(img_path):
                            img_path = os.path.join(val_dir, img_name)

                        if os.path.exists(img_path) and class_name in self.class_to_idx:
                            self.samples.append((img_path, self.class_to_idx[class_name]))
            else:
                images_dir = os.path.join(val_dir, 'images')
                if not os.path.exists(images_dir):
                    images_dir = val_dir

                for img_name in os.listdir(images_dir):
                    if img_name.lower().endswith(('.jpeg', '.jpg', '.png')):
                        img_path = os.path.join(images_dir, img_name)
                        self.samples.append((img_path, 0))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


class BrightnessTransform:
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, img):
        enhancer = ImageEnhance.Brightness(img)
        return enhancer.enhance(self.factor)

class GaussianNoiseTransform:
    def __init__(self, std):
        self.std = std

    def __call__(self, tensor):
        if self.std > 0:
            noise = torch.randn_like(tensor) * self.std
            noisy_tensor = tensor + noise
            return torch.clamp(noisy_tensor, 0, 1)
        return tensor

def get_baseline_transform():
    return transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

def get_brightness_transform(brightness_factor):
    return transforms.Compose([
        BrightnessTransform(brightness_factor),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

def get_noise_transform(noise_std):
    return transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        GaussianNoiseTransform(noise_std),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

def load_model(model_path, device):
    model = models.resnet18(pretrained=False)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 200)

    checkpoint = torch.load(model_path, map_location=device)

    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)

    model = model.to(device)
    model.eval()

    return model

def evaluate_and_log(model, dataloader, device, run_name, drift_type, drift_value,
                    baseline_accuracy=None, config=None):

    run = wandb.init(
        project=f"{config.WANDB_PROJECT}-drift",
        name=run_name,
        entity=config.WANDB_ENTITY,
        config={
            "drift_type": drift_type,
            "drift_value": drift_value,
            "baseline_accuracy": baseline_accuracy
        },
        reinit=True
    )

    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f'Evaluating {drift_type}={drift_value}'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    accuracy_drop = baseline_accuracy - accuracy if baseline_accuracy else 0.0

    # Log metrics to W&B
    wandb.log({
        "accuracy": accuracy,
        "accuracy_drop": accuracy_drop,
        "drift_type": drift_type,
        "drift_value": drift_value
    })

    # Log per-class metrics
    from sklearn.metrics import classification_report
    report = classification_report(all_labels, all_preds, output_dict=True, zero_division=0)

    for class_id, metrics in report.items():
        if class_id.isdigit():
            wandb.log({
                f"class_{class_id}_precision": metrics['precision'],
                f"class_{class_id}_recall": metrics['recall'],
                f"class_{class_id}_f1": metrics['f1-score']
            })

    # Check alert condition
    if baseline_accuracy and accuracy_drop > config.ACCURACY_DROP_THRESHOLD:
        alert_msg = f"Accuracy dropped to {accuracy:.2f}% (drop: {accuracy_drop:.2f}%) with {drift_type}={drift_value}"
        wandb.alert(
            title="Significant Accuracy Drop Detected",
            text=alert_msg,
            level=wandb.AlertLevel.WARN
        )
        print(f"\nALERT: {alert_msg}")

    print(f"{run_name}: Accuracy = {accuracy:.2f}%, Drop = {accuracy_drop:.2f}%")

    run.finish()

    return accuracy, accuracy_drop

def main():
    config = DriftConfig()

    print(f"  Entity: {config.WANDB_ENTITY}")
    print(f"  Project: {config.WANDB_PROJECT}")
    print(f"  Artifact: {config.WANDB_ARTIFACT}")
    print(f"  Device: {config.DEVICE}")

    # Download model from W&B
    success = download_model_from_wandb(config)
    if not success:
        print("\nCannot proceed without model. Exiting...")
        return

    # Load model

    model = load_model(config.MODEL_PATH, config.DEVICE)
    print("Model loaded successfully")

    # Load test dataset
    baseline_transform = get_baseline_transform()
    test_dataset = TinyImageNetDataset(config.DATA_DIR, split='test',
                                       transform=baseline_transform)

    if config.TEST_SUBSET_SIZE:
        test_indices = random.sample(range(len(test_dataset)),
                                    min(config.TEST_SUBSET_SIZE, len(test_dataset)))
        test_dataset = Subset(test_dataset, test_indices)

    print(f"Test dataset loaded: {len(test_dataset)} images")

    # BASELINE EVALUATION

    baseline_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE,
                                shuffle=False, num_workers=config.NUM_WORKERS)

    baseline_accuracy, _ = evaluate_and_log(
        model, baseline_loader, config.DEVICE,
        run_name="baseline-no-drift",
        drift_type="none",
        drift_value=0.0,
        baseline_accuracy=None,
        config=config
    )

    print(f"\nBaseline Accuracy: {baseline_accuracy:.2f}%")

    alert_threshold = baseline_accuracy - config.ACCURACY_DROP_THRESHOLD
    print(f"\nAlert Configuration:")
    print(f" Alert will trigger if accuracy drops below {alert_threshold:.2f}%")

    brightness_results = []
    noise_results = []

    # BRIGHTNESS DRIFT
    print("\n" + "="*70)
    print("SIMULATING BRIGHTNESS DRIFT")
    print("="*70)

    for brightness in config.BRIGHTNESS_FACTORS:
        run_name = f"brightness-{brightness}"

        drift_transform = get_brightness_transform(brightness)
        drift_dataset = TinyImageNetDataset(config.DATA_DIR, split='test',
                                           transform=drift_transform)
        drift_dataset = Subset(drift_dataset, test_indices)
        drift_loader = DataLoader(drift_dataset, batch_size=config.BATCH_SIZE,
                                 shuffle=False, num_workers=config.NUM_WORKERS)

        drift_accuracy, accuracy_drop = evaluate_and_log(
            model, drift_loader, config.DEVICE,
            run_name=run_name,
            drift_type="brightness",
            drift_value=brightness,
            baseline_accuracy=baseline_accuracy,
            config=config
        )

        brightness_results.append({
            'brightness': brightness,
            'accuracy': drift_accuracy,
            'drop': accuracy_drop
        })

    # NOISE DRIFT

    for noise_std in config.NOISE_LEVELS:
        run_name = f"noise-{noise_std}"

        drift_transform = get_noise_transform(noise_std)
        drift_dataset = TinyImageNetDataset(config.DATA_DIR, split='test',
                                           transform=drift_transform)
        drift_dataset = Subset(drift_dataset, test_indices)
        drift_loader = DataLoader(drift_dataset, batch_size=config.BATCH_SIZE,
                                 shuffle=False, num_workers=config.NUM_WORKERS)

        drift_accuracy, accuracy_drop = evaluate_and_log(
            model, drift_loader, config.DEVICE,
            run_name=run_name,
            drift_type="noise",
            drift_value=noise_std,
            baseline_accuracy=baseline_accuracy,
            config=config
        )

        noise_results.append({
            'noise_std': noise_std,
            'accuracy': drift_accuracy,
            'drop': accuracy_drop
        })

    # SUMMARY

    print(f"\nBaseline Accuracy: {baseline_accuracy:.2f}%")

    print("\nBrightness Drift Results:")
    for result in brightness_results:
        print(f"  Brightness {result['brightness']:.2f}: "
              f"Accuracy = {result['accuracy']:.2f}%, "
              f"Drop = {result['drop']:.2f}%")

    print("\nNoise Drift Results:")
    for result in noise_results:
        print(f"  Noise std {result['noise_std']:.2f}: "
              f"Accuracy = {result['accuracy']:.2f}%, "
              f"Drop = {result['drop']:.2f}%")

if __name__ == "__main__":
    main()

  Entity: ir2023
  Project: tiny-imagenet-assignment
  Artifact: resnet18-tiny-imagenet:latest
  Device: cuda

Logging into W&B...


wandb: Currently logged in as: 142201022 (ir2023) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact 'resnet18-tiny-imagenet:latest', 129.21MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:00.3 (446.9MB/s)
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model downloaded successfully to: outputs
Model file verified at: outputs/best_model.pth
Model loaded successfully
Val set has class folder structure with 200 classes
Test dataset loaded: 2000 images


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


...

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Evaluating noise=0.3: 100%|██████████| 32/32 [00:12<00:00,  2.66it/s]


ALERT: Accuracy dropped to 1.90% (drop: 38.90%) with noise=0.3
noise-0.3: Accuracy = 1.90%, Drop = 38.90%


accuracy,▁
accuracy_drop,▁
class_0_f1,▁
class_0_precision,▁
class_0_recall,▁
class_100_f1,▁
class_100_precision,▁
class_100_recall,▁
class_101_f1,▁
class_101_precision,▁
+593,...



Baseline Accuracy: 40.80%

Brightness Drift Results:
  Brightness 0.30: Accuracy = 28.95%, Drop = 11.85%
  Brightness 0.50: Accuracy = 37.90%, Drop = 2.90%
  Brightness 0.70: Accuracy = 40.15%, Drop = 0.65%
  Brightness 1.00: Accuracy = 40.80%, Drop = 0.00%
  Brightness 1.30: Accuracy = 39.90%, Drop = 0.90%
  Brightness 1.50: Accuracy = 37.55%, Drop = 3.25%
  Brightness 1.70: Accuracy = 34.90%, Drop = 5.90%

Noise Drift Results:
  Noise std 0.00: Accuracy = 40.80%, Drop = 0.00%
  Noise std 0.05: Accuracy = 37.55%, Drop = 3.25%
  Noise std 0.10: Accuracy = 24.35%, Drop = 16.45%
  Noise std 0.15: Accuracy = 13.60%, Drop = 27.20%
  Noise std 0.20: Accuracy = 6.40%, Drop = 34.40%
  Noise std 0.25: Accuracy = 3.05%, Drop = 37.75%
  Noise std 0.30: Accuracy = 1.90%, Drop = 38.90%
